In [ ]:
!pip install pyspark==3.5.0 findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=fa7d6d3b8868eedbf55b30775e26c34b6f76c8469d5c624e28e89d545ce96945
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install delta-spark

In [ ]:
pyspark --packages com.databricks:spark-xml_2.12:0.15.0

SyntaxError: invalid syntax (<ipython-input-7-b577ed9b5509>, line 1)

In [ ]:
import findspark
findspark.init()


In [ ]:
from pyspark.sql import SparkSession
from delta import *

def _create_delta_spark():
  from pyspark.sql import SparkSession
  from delta import configure_spark_with_delta_pip
  builder = SparkSession.builder.appName("DeltaLakeApp") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
  .config("spark.jars.packages","io.delta:delta-core_2.12:2.0.0")\
  .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")
  return configure_spark_with_delta_pip(builder).getOrCreate()

spark = _create_delta_spark()

In [ ]:
movies_metadata_df = spark.read.csv("/content/sample_data/movies_metadata.csv", header=True, inferSchema=True)

In [ ]:
movies_metadata_df.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [ ]:
movies_metadata_df.write.format('xml').save('/content/sample_data/movies_metadata/')

In [ ]:
movies_metadata_df.count()

7667

In [ ]:
movies_metadata_df = spark.read.format('delta').load('/content/sample_data/movies_metadata/')

In [ ]:
ratings_df = spark.read.csv("/content/sample_data/ratings.csv", header=True, inferSchema=True)

In [ ]:
links_df = spark.read.csv("/content/sample_data/links.csv", header=True, inferSchema=True)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("XMLExample").getOrCreate()

df = spark.read.format("xml")\
               .option("rowTag", "movie")\
               .load("movies.xml")
